# Подготовка данных

- уменьшение размера выборки пользователей для ускорения обучения модели

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from recsys import config

2024-12-16 23:11:14.729 | INFO     | recsys.config:<module>:11 - PROJ_ROOT path is: /Users/egortarasov/uni/7 семестр/nlp/recsys


In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
ARTICLES_CNT = 1000

## Выбираем 1000 самых популярных лотов из транзакций

In [3]:
init_transactions = pd.read_csv(config.RAW_DATA_DIR / "transactions_train.csv")

In [4]:
top_article_ids_counts = init_transactions["article_id"].value_counts()[:ARTICLES_CNT]

In [5]:
top_article_ids_counts

article_id
706016001    50287
706016002    35043
372860001    31718
610776002    30199
759871002    26329
             ...  
512251004     3269
562245097     3268
575347014     3264
559616014     3259
625939003     3258
Name: count, Length: 1000, dtype: int64

In [6]:
top_article_ids = top_article_ids_counts.to_dict().keys()

In [7]:
len(init_transactions)

31788324

In [8]:
transactions = init_transactions[init_transactions["article_id"].isin(top_article_ids)]

In [9]:
len(transactions)

5772706

In [10]:
transactions.to_csv(config.PROCESSED_DATA_DIR / "tx.csv")

In [11]:
init_customers = pd.read_csv(config.RAW_DATA_DIR / "customers.csv")
len(init_customers)

1371980

In [12]:
customers = init_customers[init_customers["customer_id"].isin(transactions["customer_id"])]
len(customers)

902242

In [13]:
customers.to_csv(config.PROCESSED_DATA_DIR / "customers.csv")

In [14]:
init_articles = pd.read_csv(config.RAW_DATA_DIR / "articles.csv")
len(init_articles)

105542

In [15]:
articles = init_articles[init_articles["article_id"].isin(top_article_ids)]
len(articles)

1000

In [16]:
articles.to_csv(config.PROCESSED_DATA_DIR / "articles.csv")

In [17]:
# Загрузка данных
transactions = pd.read_csv(config.PROCESSED_DATA_DIR / "tx.csv")

# Преобразование столбца t_dat в формат временной метки
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])

# Сортировка по значению t_dat
transactions = transactions.sort_values(by="t_dat")

# Группировка по customer_id и создание списка покупок
grouped_transactions = (
    transactions.groupby("customer_id")["article_id"].apply(list).reset_index()
)

# Переименование столбца
grouped_transactions.columns = ["customer_id", "articles"]

# Сохранение отфильтрованного датасета
grouped_transactions.to_csv(config.PROCESSED_DATA_DIR / "grouped_transactions.csv", index=False)